# misc

In [ ]:
import os, importlib
import conf, utils, analysis_utils
importlib.reload(conf)
importlib.reload(utils)

from transformers import AutoTokenizer
from conf import config
from utils import *
from analysis_utils import *

In [ ]:
from math_verify import parse, verify

# Parse the gold and answer
# If you know that gold will only contain latex or expr (no latex env), use
# parse(gold, extraction_config=[LatexExtractionConfig()]) or parse(gold, extraction_config=[ExprExtractionConfig()])
t1="72"
t2="72<im_end>"

gold = parse(t1)
answer = parse(t2)

score=verify(gold, answer)
print(score)

In [ ]:
scratch_disk_dir=os.environ.get("scratch_disk_dir")
data_dir="data/DeepSeek-R1-Distill-Qwen-1.5B_gsm8k_06-15_16-52-17"

config,SAVE_DATA=read_save_data(data_dir)

import pandas as pd

# Create empty dataframe with token budgets as columns and sample indices as rows
token_budgets = sorted(list(SAVE_DATA.keys()))
n_samples = len(SAVE_DATA[token_budgets[0]].keys()) - 1  # -1 to exclude 'metrics' key
df = pd.DataFrame(index=range(n_samples), columns=token_budgets)

# Fill dataframe with text data
for token_budget in token_budgets:
    for sample_idx in range(n_samples):
        df.loc[sample_idx, token_budget] = SAVE_DATA[token_budget][sample_idx]['text']

with open('data.html', 'w') as f:
    f.write(df.to_html())


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir=os.environ.get("HF_CACHE_PATH"))

# Print chat templates for various models
models = [
    "meta-llama/Llama-2-7b-chat-hf",
    "mistralai/Mistral-7B-Instruct-v0.2", 
    "google/gemma-7b-it",
    "deepseek-ai/deepseek-coder-7b-instruct",
    "Qwen/Qwen1.5-7B-Chat",
    "microsoft/phi-2",
    "simplescaling/s1.1-32B",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
]

for model in models:
    try:
        tokenizer = AutoTokenizer.from_pretrained(model, cache_dir=os.environ.get("HF_CACHE_PATH"))
        print(f"Model: {model}")
        print(f"Special tokens: {tokenizer.special_tokens_map}")
        print(f"Pad token: {tokenizer.pad_token}")
        print(f"EOS token: {tokenizer.eos_token}")
        print(f"BOS token: {tokenizer.bos_token}")
    except:
        print(f"Error loading tokenizer for {model}")



In [ ]:
import analysis_utils
importlib.reload(analysis_utils)
from analysis_utils import *


path="data/quick_save"
config,SAVE_DATA=read_save_data(path)

completions_text_view(SAVE_DATA,token_budget=512)


In [ ]:
import torch 

from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B", cache_dir=os.environ.get("HF_CACHE_PATH"))

def no_right_or_middle_padding(input_ids, tokenizer):
    # Convert to tensor if not already
    if not isinstance(input_ids, torch.Tensor):
        input_ids = torch.tensor(input_ids)
    
    # Get positions of all pad tokens
    pad_positions = (input_ids == tokenizer.pad_token_id).nonzero()
    
    if len(pad_positions) == 0:
        return True
        
    # Get positions of all non-pad tokens
    nonpad_positions = (input_ids != tokenizer.pad_token_id).nonzero()
    
    # Check that all pad tokens come before all non-pad tokens
    return all(p < nonpad_positions[0].item() for p in pad_positions)


def no_right_padding(input_ids, tokenizer):
    # Convert to tensor if not already
    if not isinstance(input_ids, torch.Tensor):
        input_ids = torch.tensor(input_ids)
    
    # Get positions of all pad tokens
    pad_positions = (input_ids == tokenizer.pad_token_id).nonzero()
    
    if len(pad_positions) == 0:
        return True
        
    # Get positions of all non-pad tokens
    nonpad_positions = (input_ids != tokenizer.pad_token_id).nonzero()
    
    # Check that all pad tokens come after all non-pad tokens
    return all(p > nonpad_positions[-1].item() for p in pad_positions)

# Test cases
test_seqs = [
    torch.tensor([1, 2, 3, 4, 5]),  # No padding
    torch.tensor([1, 2, tokenizer.pad_token_id, tokenizer.pad_token_id]),  # Right padding
    torch.tensor([tokenizer.pad_token_id, 1, 2, 3]),  # Left padding
    torch.tensor([1, tokenizer.pad_token_id, 2, 3])  # Middle padding
]

for i, seq in enumerate(test_seqs):
    print(f"Sequence {i}: {no_right_or_middle_padding(seq, tokenizer)}")
    print(f"Sequence {i}: {no_right_padding(seq, tokenizer)}")


In [ ]:
from transformers import AutoTokenizer

tokenizer_names = [
    #"EleutherAI/pythia-70m",
    #"Qwen/QwQ-32B", 
    "google/gemma-2-2b-it",

    #"open-thoughts/OpenThinker-32B"
]

for name in tokenizer_names:
    print(f"\n--- Inspecting tokenizer: {name} ---")
    tokenizer = AutoTokenizer.from_pretrained(name)

    chat_template = getattr(tokenizer, "chat_template", None)
    if chat_template is not None:
        print(f"Chat template found for {name}.")
    else:
        print(f"No chat template found for {name}.")

    print(f"BOS token: {tokenizer.bos_token}")
    print(f"EOS token: {tokenizer.eos_token}") 
    print(f"PAD token: {tokenizer.pad_token}")

    # Example chat messages (if chat template exists)
    if chat_template is not None:
        messages = [
            {"role": "user", "content": "Hello, how are you?"},
            {"role": "assistant", "content": "I'm good, thank you! How can I help you today?"},
        ]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)
        print(f"\nChat prompt: {prompt}")
        tokens = tokenizer(prompt)
        print(f"Tokenized chat prompt: {tokens['input_ids']}")
        print(f"Decoded: {tokenizer.decode(tokens['input_ids'])}")

        print('\n\n\n')
    else:
        # If no chat template, just tokenize as normal
        prompt = "Hello, how are you?"
        tokens = tokenizer(prompt)
        print(f"Tokenized prompt: {tokens['input_ids']}")
        print(f"Decoded: {tokenizer.decode(tokens['input_ids'])}")
        print('\n\n\n')
    

   
#find end of input ids 
tokens = tokenizer.tokenize(prompt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
for toks,ids in list(zip(tokens,token_ids)):
    print(f"{toks}: {ids}")

In [ ]:
path = "prompts/trace_decomposition.txt"
with open(path, "r") as f: 
    output = f.read()

splitter_string = "\n\n\n\n"

input_trace = "SAMPLE_INPUT_TRACE"
input_trace_identifier = "INSERT_REASONING_TRACE"
input_prompt = "SAMPLE_INPUT_PROMPT"
input_prompt_identifier = "INSERT_INPUT_PROMPT"

SYSTEM_PROMPT = output.split("SYSTEM_PROMPT=")[1].split("\n")[0].strip()
USER_PROMPT = output.split("USER_PROMPT")[1].split("=")[-1].strip()


USER_PROMPT = USER_PROMPT.replace(input_trace_identifier, input_trace)
USER_PROMPT = USER_PROMPT.replace(input_prompt_identifier, input_prompt)

print(SYSTEM_PROMPT)
print(USER_PROMPT)

In [ ]:
import pandas as pd

# Create dummy data: 3 rows, 4 columns, each cell is a tuple (a, b)
data = [
    [(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')],
    [(5, 'e'), (6, 'f'), (7, 'g'), (8, 'h')],
    [(9, 'i'), (10, 'j'), (11, 'k'), (12, 'l')]
]

columns = ['Col1', 'Col2', 'Col3', 'Col4']
index = ['Row1', 'Row2', 'Row3']

df = pd.DataFrame(data, columns=columns, index=index)
print(df)


# vllm

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer

pythia_tokenizer=AutoTokenizer.from_pretrained("EleutherAI/pythia-70m", cache_dir=os.environ.get("HF_CACHE_PATH"))

# Add special tokens
pythia_tokenizer.add_special_tokens({
    'additional_special_tokens': ['<WACKY1>', '<WACKY2>', '<SUPER_WACKY>']
})

# Create temp dir and save
tmp_dir = "tmp_tokenizer"
os.makedirs(tmp_dir, exist_ok=True)
pythia_tokenizer.save_pretrained(tmp_dir)


try:
    vllm_model = LLM(model="EleutherAI/pythia-70m",
    tokenizer = "tmp_tokenizer",
    trust_remote_code=True)
    print("VLLM available")
except Exception as e:
    print(f"VLLM not available: {e}")

# Basic VLLM generation example
sampling_params = SamplingParams(
    temperature=0.7,
    top_p=0.95,
    max_tokens=100
)

prompts = [
    "What is 2+2?",
    "Explain quantum computing in one sentence."
]

outputs = vllm_model.generate(prompts, sampling_params)

# dataset checks

In [ ]:
import importlib
import utils
import sys 



#clean wipe
if "utils" in sys.modules:
    del sys.modules["utils"]
if "utils.dataset_utils" in sys.modules:
    del sys.modules["utils.dataset_utils"]



import utils
import utils.dataset_utils


In [ ]:
dataset_names=["math-ai/aime25","Maxwell-Jia/AIME_2024","Idavidrein/gpqa"]
n_samples=10

for dataset_name in dataset_names:
    dataset_setup = utils.DatasetSetup(dataset_name,n_samples=n_samples)
    dataset = dataset_setup.load_standardized_dataset()
    print(f"\n{dataset_name}:")
    print(f"Number of samples: {len(dataset)}")
    print("\nSample questions and answers:")
    for i, sample in enumerate(dataset):
        print(f"\nSample {i+1}:")
        print(f"Question: {sample['question']}")
        print(f"Answer: {sample['answer']} (type: {type(sample['answer'])})")

# models

In [ ]:
from transformers import AutoTokenizer
import torch


tokenizer_list = [
    #"EleutherAI/pythia-70m",
    #"deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "google/gemma-2-2b-it",
    "Qwen/Qwen2.5-3B-Instruct",
    "simplescaling/s1.1-32B",

]

if 0: 
    for tokenizer in tokenizer_list:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer)
        print(f"\nTokenizer: {tokenizer.name_or_path}")
        print("Special tokens:")
        for token_name, token_value in tokenizer.special_tokens_map.items():
            print(f"{token_name}: {token_value}")
        print("---")

def get_end_of_input_idxs(input_ids, tokenizer):
    """
    Gets 'end of input' token indices for each sequence in the batch.
    Returns tensor of indices where the end_of_input pattern ends.
    """
    
    # Fix the condition logic - use 'in' properly
    if "Qwen2.5" in tokenizer.name_or_path or "simplescaling" in tokenizer.name_or_path:
        end_of_input_ids = torch.tensor([151644, 77091, 198])  # '<|im_start|>assistant\n'
    elif "gemma-2" in tokenizer.name_or_path:
        end_of_input_ids = torch.tensor([106, 2516, 108])  # '<start_of_turn>model\n'
    else:
        raise ValueError(f"Tokenizer {tokenizer.name_or_path} not supported for force continuation")
    
    end_of_input_idxs = []
    pattern_len = len(end_of_input_ids)
    
    for seq in input_ids:
        found = False
        # Use sliding window to find pattern
        for i in range(len(seq) - pattern_len + 1):
            if torch.all(seq[i:i+pattern_len] == end_of_input_ids):
                # Return the index AFTER the pattern (where generation starts)
                end_of_input_idxs.append(list(range(i,i+pattern_len)))
                found = True
                break
        
        if not found:
            # If pattern not found, assume entire sequence is input
            end_of_input_idxs.append(len(seq))
            
    return end_of_input_idxs

In [ ]:
# Create some sample input sentences
input_sentences = [
    "What is the capital of France?",
]

# Apply chat template and compare for each tokenizer
for tokenizer_name in tokenizer_list:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    print(f"\nTokenizer: {tokenizer.name_or_path}")
    
    for sentence in input_sentences:
        print(f"\nOriginal input: {sentence}")
        
        # Get base tokenization
        base_tokens = tokenizer.encode(sentence)
        print(f"Base tokenization: {base_tokens}")
        print(f"Decoded base tokens: {tokenizer.decode(base_tokens)}")
        
        # Apply chat template if available
        if tokenizer.chat_template is not None:
            formatted = tokenizer.apply_chat_template([{"role": "user", "content": sentence}], tokenize=False,add_generation_prompt=True)
            formatted_tokens = tokenizer.encode(formatted)
            print(f"Chat template: {formatted}")
            print(f"Chat template tokens: {formatted_tokens}")
            print(f"Decoded chat template: {tokenizer.decode(formatted_tokens)}")
            
            # Get end of input indices
            eoi_idxs = get_end_of_input_idxs(torch.tensor([formatted_tokens]), tokenizer)
            print(f"End of input index: {eoi_idxs}")
            print(f"End of input token: {formatted_tokens[eoi_idxs]}")
            print(f"Decoded end of input: {tokenizer.decode(formatted_tokens[eoi_idxs])}")
        else:
            print("No chat template available")
        print("---")

In [ ]:
# Print chat templates for each tokenizer
for tokenizer_name in tokenizer_list:
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    print(f"\nTokenizer: {tokenizer.name_or_path}")
    print(f"Chat template: {tokenizer.chat_template}")


# dummy data

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import importlib
import random


import utils
importlib.reload(utils)
importlib.reload(utils.analysis_utils)
from utils.analysis_utils import *

np.random.seed(42)
random.seed(42)

In [ ]:
dummy_data = generate_dummy_data([128, 256, 512, 1024, 2048], 5, 3)

metrics=["score","probs","entropy"]
utils.plot_data(n_samples=2,SAVE_DATA=dummy_data,metrics=metrics)


In [ ]:
# Generate sample data
n_ys=12
x = np.linspace(0, 10, 100)
ys = []
labels = []

# Generate n_ys different curves
for i in range(n_ys):
    curve_type = i % 3  # Cycle through 3 different curve types
    if curve_type == 0:
        y = np.sin(x + i/2) + 0.1*np.random.randn(len(x))
        labels.append(f'sin(x+{i/2:.1f})')
    elif curve_type == 1:
        y = ((x+i)**2)/100 + 0.1*np.random.randn(len(x))
        labels.append(f'(x+{i})^2')
    else:
        y = np.exp(-(x+i)/5) + 0.1*np.random.randn(len(x))
        labels.append(f'exp(-(x+{i})/5)')
    ys.append(y)

# Plot curves
plt.figure(figsize=(12,4))
plt.subplot(121)
for y, label in zip(ys, labels):
    plt.plot(x, y, label=label)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)

# Plot distribution of differences for all curves
plt.subplot(122)
for y, label in zip(ys, labels):
    plt.hist(np.diff(y), bins=20, density=True, alpha=0.5, label=label)
plt.xlabel('Difference')
plt.ylabel('Density')
plt.title('Distribution of Differences')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.cluster import KMeans

ys=np.array(ys)
diffs=np.diff(ys,axis=1) 
n_clusters=3

# Reshape diffs to 2D array for clustering
diffs_reshaped = diffs.reshape(diffs.shape[0], -1)

# Perform k-means clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(diffs_reshaped)

# Plot results
plt.figure(figsize=(12,4))

# Plot original curves colored by cluster
plt.subplot(121)
for i, (y, label) in enumerate(zip(ys, labels)):
    plt.plot(x, y, label=f'Cluster {clusters[i]}', alpha=0.7)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Curves Colored by Difference Cluster')
plt.legend()
plt.grid(True)

# Plot cluster centers
plt.subplot(122)
centers = kmeans.cluster_centers_.reshape(n_clusters, -1)
for i, center in enumerate(centers):
    plt.plot(np.arange(len(center)), center, label=f'Cluster {i} Center')
plt.xlabel('Position')
plt.ylabel('Difference')
plt.title('Cluster Centers')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


# results check

In [ ]:
import os

# Force set the environment variable
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE' #fix this bug

import importlib
import utils
import conf

importlib.reload(conf)
importlib.reload(utils)


In [ ]:
path="/Users/iyngkarrankumar/Documents/Edinburgh MScR/Easter (Q3)/random_experiments/budget_forcing_emergence/results_data/DeepSeek-R1-Distill-Qwen-1.5B_gsm8k_07-05_19-55-37"

config,SAVE_DATA=utils.read_save_data(path)

In [ ]:
utils.token_budget_text_view(SAVE_DATA)

# gdrive check

In [ ]:

import utils, config, utils.analysis_utils as analysis_utils

path = "/Users/iyngkarrankumar/Google Drive/My Drive/results_data"

config,SAVE_DATA = 

# tokenization check

In [ ]:
import os, importlib
from transformers import AutoTokenizer

model_names = [
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
]

test_text = "Quick brown fox jumped over the lazy dog"

for model_name in model_names:
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        print(f"Model: {model_name}")
        print("Special tokens:", tokenizer.special_tokens_map)
        if hasattr(tokenizer, "chat_template") and tokenizer.chat_template is not None:
            print("Chat template found.")
            try:
                chat_encoded = tokenizer.apply_chat_template(
                    [{"role": "user", "content": test_text}],
                    tokenize=False,
                    add_generation_prompt=True if "llama" in model_name.lower() or "qwen" in model_name.lower() else False
                )
                print("Chat template output:", chat_encoded)
            except Exception as e:
                print(f"Error applying chat template: {e}")
        else:
            print("No chat template found.")
        print("-" * 40)
    except Exception as e:
        print(f"Could not load tokenizer for {model_name}: {e}")


# analysis debug

In [ ]:
import os 
import importlib
import utils
import utils


importlib.reload(utils)
importlib.reload(utils.analysis_utils)
all_paths_dir = "results_data/multi_run_results"

paths = os.listdir(all_paths_dir)
paths = [os.path.join(all_paths_dir, path) for path in paths]
save_dir = "analysis_outputs"
analysis_workflow = utils.analysis_utils.AnalysisWorkflow(paths=paths, results_save_dir=save_dir)

#analysis_workflow.aggregate_plots(SHOW=False)]

model_name = "deepseek"
dataset_name = "aime_2024"
tables = analysis_workflow.get_metric_tables(model_name=model_name,dataset_name=dataset_name)

In [ ]:
import numpy as np 

x = np.array([0.        , 0.        , 0.        , 0.01333333, 0.2       ])
y = np.array([0.00163385, 0.00186161, 0.00218855, 0.01744281, 0.19760427])

mu_x = np.mean(x)
mu_y = np.mean(y)

x_mu_dist = x-mu_x
y_mu_dist = y-mu_y
x_mu_dist_sq = x_mu_dist**2
y_mu_dist_sq = y_mu_dist**2

mu_dist_product = x_mu_dist * y_mu_dist


numerator = np.sum(mu_dist_product)
denominator = np.sqrt(np.sum(x_mu_dist_sq) * np.sum(y_mu_dist_sq))

pearson_corr = numerator/denominator





